In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense,LSTM,Embedding,Dropout
from keras.models import Sequential,Model

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/futureprediction/items.csv
/kaggle/input/futureprediction/sample_submission.csv
/kaggle/input/futureprediction/item_categories.csv
/kaggle/input/futureprediction/sales_train.csv
/kaggle/input/futureprediction/shops.csv
/kaggle/input/futureprediction/test.csv


In [2]:
item_cat=pd.read_csv('/kaggle/input/futureprediction/item_categories.csv')
items=pd.read_csv('/kaggle/input/futureprediction/items.csv')
shops=pd.read_csv('/kaggle/input/futureprediction/shops.csv')
sample_sub=pd.read_csv('/kaggle/input/futureprediction/sample_submission.csv')

In [3]:
df_train=pd.read_csv('/kaggle/input/futureprediction/sales_train.csv')

In [4]:
df_train['date']=pd.to_datetime(df_train['date'],format ='%d.%m.%Y')

In [5]:
dataset=df_train.pivot_table(index=['shop_id','item_id'],values=['item_cnt_day'],columns=['date_block_num'],fill_value=0,aggfunc='sum')
dataset.reset_index(inplace=True)
dataset

shop_id item_id item_cnt_day                               
date_block_num                            0   1  2  3  4  5  6  7  8  9 10
0                    0      30            0  31  0  0  0  0  0  0  0  0  0
1                    0      31            0  11  0  0  0  0  0  0  0  0  0
2                    0      32            6  10  0  0  0  0  0  0  0  0  0
3                    0      33            3   3  0  0  0  0  0  0  0  0  0
4                    0      35            1  14  0  0  0  0  0  0  0  0  0
...                ...     ...          ...  .. .. .. .. .. .. .. .. .. ..
235421              59   22150            0   0  0  0  0  0  1  1  0  0  0
235422              59   22151            2   0  1  0  0  0  0  0  0  0  0
235423              59   22154            1   0  0  0  0  0  0  0  0  0  0
235424              59   22155            0   0  0  0  0  0  1  0  0  0  0
235425              59   22167            0   0  0  0  0  0  0  0  0  1  0

[235426 rows x 13 columns]

In [6]:
df_test=pd.read_csv('/kaggle/input/futureprediction/test.csv')

In [7]:
dataset=pd.merge(df_test,dataset,on=['item_id','shop_id'],how='left')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [8]:
dataset.fillna(0,inplace=True)
dataset.isnull().sum().sum()

0

In [9]:
dataset.drop(['shop_id','item_id','ID'],inplace=True,axis=1)
dataset

,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)","(item_cnt_day, 10)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
214195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
x_train=np.expand_dims(dataset.values[:,:-1],axis=2)
y_train=dataset.values[:,-1:]
y_train[y_train<0]=0
x_test=np.expand_dims(dataset.values[:,1:],axis=2)
print(x_train.shape,y_train.shape,x_test.shape)

(214200, 10, 1) (214200, 1) (214200, 10, 1)


In [11]:
lstm=Sequential()
lstm.add(LSTM(units=128,input_shape=(33,1),return_sequences=True ))
lstm.add(LSTM(units=128,input_shape=(33,1),return_sequences=True ))


lstm.add(Dropout(0.4))
lstm.add(LSTM(units=128,input_shape=(33,1)))
lstm.add(Dense(1))
lstm.compile(loss='mse',optimizer='adam',metrics=['mean_squared_error'])

2022-12-04 18:16:16.420569: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [12]:
lstm.fit(x_train,y_train,batch_size=4096,epochs=5)

2022-12-04 18:16:17.379705: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
53/53 [==============================] - 64s 1s/step - loss: 9.8558 - mean_squared_error: 9.8558
Epoch 2/5
53/53 [==============================] - 60s 1s/step - loss: 9.6322 - mean_squared_error: 9.6322
Epoch 3/5
53/53 [==============================] - 60s 1s/step - loss: 9.5111 - mean_squared_error: 9.5111
Epoch 4/5
53/53 [==============================] - 59s 1s/step - loss: 9.4868 - mean_squared_error: 9.4868
Epoch 5/5
53/53 [==============================] - 59s 1s/step - loss: 9.4256 - mean_squared_error: 9.4256


In [13]:
submission_file=lstm.predict(x_test)
submission_file=submission_file.clip(0.20)
submission_trp=pd.DataFrame({'ID':df_test['ID'],'item_cnt_month':submission_file.ravel()})
submission_trp.to_csv('submission(-ve removed).csv',index=False)
submission_trp

,ID,item_cnt_month
0,0,0.2
1,1,0.2
2,2,0.2
3,3,0.2
4,4,0.2
...,...,...
214195,214195,0.2
214196,214196,0.2
214197,214197,0.2
214198,214198,0.2
